# Lab-3:
# Exercise 2:

> Importing necessary libraries:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from textblob import TextBlob
import string
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, roc_curve
sb.set_theme()

> Loading the dataset:

In [2]:
data = pd.read_csv('Womens Clothing E-Commerce Reviews.csv\Womens Clothing E-Commerce Reviews.csv')
data.head(2)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses


#### Question-1:</br>  Preprocessing: </br> &emsp; a.  Find any null values are present or not, If present remove those data. </br> &emsp; b.  Remove the data that have less than 5 reviews. </br> &emsp; c.  Clean the data and remove the special characters and replace the contractions with its &emsp; expansion. Convert the uppercase character to lower case. Also, remove the punctuations.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


In [4]:
data.shape

(23486, 11)

In [5]:
# data = data.drop(columns=['Title'])

In [6]:
data.isnull().sum()

Unnamed: 0                    0
Clothing ID                   0
Age                           0
Title                      3810
Review Text                 845
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64

In [7]:
data = data.dropna()
data.shape

(19662, 11)

In [8]:
data.isnull().sum()

Unnamed: 0                 0
Clothing ID                0
Age                        0
Title                      0
Review Text                0
Rating                     0
Recommended IND            0
Positive Feedback Count    0
Division Name              0
Department Name            0
Class Name                 0
dtype: int64

In [9]:
data['Rating'] = 0

for cloth in data['Clothing ID'].unique():
    data.loc[data['Clothing ID']==cloth, 'Rating'] = data['Clothing ID'].value_counts()[cloth]
    
data = data[data.Rating>=5]
data.shape

(18401, 11)

> Text for contractions:

In [10]:
contractions = {
"a'ight":"alright",
"ain't":"are not",
"amn't":"am not",
"aren't":"are not",
"can't":"cannot",
"'cause": "because",
"could've":"could have",
"couldn't":"could not",
"couldn't've":"could not have",
"daren't":"dare not",
"daresn't":"dare not",
"dasn't":"dare not",
"didn't":"did not",
"doesn't":"does not",
"don't":"do not",
"everybody's":"everybody is",
"everyone's":"everyone is",
"giv'n":"given",
"gonna":"going to",
"gon't":"go not", 
"gotta":"got to",
"hadn't":"had not",
"had've":"had have",
"hasn't":"has not",
"haven't":"have not",
"he'd":"he had", 
"he'll":"he will",
"he's":"he is",
"here's":"here is",
"how'd":"how did",
"how'll":"how will",
"how're":"how are",
"how's":"how is",
"I'd":"I had",
"I'd've":"I would have",
"I'd'nt":"I would not",
"I'd'nt've":"I would not have",
"I'll":"I will",
"I'm":"I am",
"I've":"I have",
"isn't":"is not",
"it'd":"it would",
"it'll":"it will",
"it's":"it is",
"let's":"let us",
"ma'am":"madam",
"mayn't":"may not",
"may've":"may have",
"mightn't":"might not",
"might've":"might have",
"mustn't":"must not",
"mustn't've":"must not have",
"must've":"must have",
"needn't":"need not",
"needn't've":"need not have",
"o'clock":"of the clock",
"oughtn't":"ought not",
"oughtn't've":"ought not have",
"shan't":"shall not",
"she'd":"she would",
"she'll":"she will",
"she's":"she is",
"should've":"should have",
"shouldn't":"should not",
"shouldn't've":"should not have",
"somebody's":"somebody is",
"someone's":"someone is",
"something's":"something is",
"so're":"so are",
"so’s":"so is",
"so’ve":"so have",
"that'll":"that will",
"that're":"that are",
"that's":"that is",
"that'd":"that would",
"there'd":"there would",
"there'll":"there will",
"there're":"there are",
"there's":"there is",
"these're":"these are",
"these've":"these have",
"they'd":"they would",
"they'll":"they will",
"they're":"they are",
"they've":"they have",
"this's":"this is",
"those're":"those are",
"those've":"those have",
"to've":"to have",
"wasn't":"was not",
"we'd":"we would",
"we'd've":"we would have",
"we'll":"we will",
"we're":"we are",
"we've":"we have",
"weren't":"were not",
"what'd":"what did",
"what'll":"what will",
"what're":"what are",
"what's":"what is",
"what've":"what have",
"when's":"when is",
"where'd":"where did",
"where'll":"where will",
"where're":"where are",
"where's":"where is",
"where've":"where have",
"which'd":"which would",
"which'll":"which will",
"which're":"which are",
"which's":"which is",
"which've":"which have",
"who'd":"who would",
"who'd've":"who would have",
"who'll":"who will",
"who're":"who are",
"who's":"who is",
"who've":"who have",
"why'd":"why did",
"why're":"why are",
"why's":"why is",
"won't":"will not",
"would've":"would have",
"wouldn't":"would not",
"wouldn't've":"would not have",
"y'at":"you at",
"yes’m":"yes madam",
"you'd":"you would",
"you'll":"you will",
"you're":"you are",
"you've":"you have"}

> Function for converting contractions to expansions:

In [11]:
def cont_to_exp(x):
    if type(x) is str:
        x = x.replace('\\','')
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
        return x
    else:
        return x

> Functions Upper to Lower case:

In [12]:
def upper_to_lower(x):
    if type(x) is str:
        return x.lower()
    else:
        return x

> Functions to remove punctuations:

In [13]:
def remove_punctuations(text):
    if type(text) is str:
        for p in string.punctuation:
            text = text.replace(p, '')
        return text
    else:
        return text 

> Function Remove Special Characters:

In [14]:
def remove_special_chars(text):
    special_chars = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    if type(text) is str:
        for p in special_chars:
            text = text.replace(p, '')
        return text
    else:
        return text

In [15]:
data['Review Text'] = data['Review Text'].apply(lambda x:cont_to_exp(x))

In [16]:
data['Review Text'] = data['Review Text'].apply(lambda x:upper_to_lower(x))

In [17]:
data['Review Text'] = data['Review Text'].apply(lambda x:remove_punctuations(x))

In [18]:
data['Review Text'] = data['Review Text'].apply(lambda x:remove_special_chars(x))

In [19]:
data['polarity_review'] = data['Review Text'].apply(lambda x: TextBlob(x).sentiment.polarity)

#### Question-2: Separate the columns into dependent and independent variables (or features and labels). Then you split those variables into train and test sets (80:20).

In [20]:
data = data.drop(columns=['Unnamed: 0', 'Clothing ID', 'Review Text', 'Title', 'Department Name', 'Division Name', 'Class Name'])

In [21]:
x = data.drop(columns=['Recommended IND'])
y = data['Recommended IND']

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=105)
numerical_cols = [cname for cname in x.columns if x[cname].dtype in ['int64', 'float64']]
categorical_cols = [cname for cname in x.columns if x[cname].dtype == 'object']

#### Question-3: Apply the Naïve Bayes Classification Algorithm on Sentiment category to predict if item is recommended

In [23]:
numerical_transformer = Pipeline(steps = [
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('vectorizer', CountVectorizer(stop_words='english', ngram_range=(1,2), max_features=10)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
      ])

In [24]:
model = GaussianNB()
pipe = Pipeline(steps=[('preprocessor', preprocessor), 
                              ('model', model)])

In [25]:
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)
proba = pipe.predict_proba(x_test)

#### Question-4: Tabulate accuracy in terms of precision, recall and F1 score.

In [26]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.41      0.09      0.15       680
           1       0.83      0.97      0.89      3001

    accuracy                           0.81      3681
   macro avg       0.62      0.53      0.52      3681
weighted avg       0.75      0.81      0.76      3681

